# Script to match the relevant tables for 4Hi-q

## Requirements

To be able to run this, the following requirements need to be met:

- A `python 3.10` environment, including the `astropy`, `astroquery`, `scipy` and standard modules
- The `function package` and the `catalogues` in the same directory as this script.

## Structure

This script consists of a few separate parts to handle the catalogue data, divided into the following sections:

- Querying for the catalogue data and writing backups to disk, requiring the following:
  - A `region` specified in the beginning, used for RA and DEC constraints
  - An `agn_cat`, providing RA, DEC and probability information on the AGNs
  - The `sweep_cat` catalogues for the region in question (TODO: Use the LS DR query interface)
  - The `vhs_cat` is obtained by querying the Vista Science Archive
- Matching the catalogues, also writing a backup.
  - The matching is performed on the respective RA and DEC coordinates (TODO: Use the advanced constraints described in my thesis)
  - A match via the `cds` service is performed with the GALEX catalogue
- Processing the catalogues (unit conversions and reddening correction), including a split into *pointlike* and *extended*.
- Collapsing everything into a `LePhare` input file
  
Note: After running this script successfully, the photometric redshift routine can be carried out using `run_lephare.ipynb`

## Citations

- Please see `other/citations.md` for any citations/acknowledgements that might be required when using this script.

All of the functions used for these routines can either be found directly in this notebook or in the `function_package` directory.

In [1]:
import os
import function_package as fp
import logging
logging.getLogger().setLevel("INFO")

logging.info("Modules loaded successfully")

INFO:root:Modules loaded successfully


Could not import regions, which is required for some of the functionalities of this module.


In [2]:
# Set up the directories necessary
fp.generate_all_filepaths()

In [3]:
# Define the region to constrain everything to:
# The eFEDs field is dec < 6.2 & dec > -3.2 & ra < 146.2 & ra > 126.
ra_min, ra_max, dec_min, dec_max = 125, 130, -5, 0
REGION = fp.Region(ra_min, ra_max, dec_min, dec_max)
logging.info(REGION)
REGION.save_to_disk()

INFO:root:Region constrained to 125.0000 <= RA <= 130.0000 and -5.0000 <= DEC <= 0.0000.
This corresponds to a linear (!) size of 25 deg^2


## Loading the tables

In this section, the tables are loaded.

In [4]:
SHU_TABLE = fp.load_and_clean_opt_agn_shu(REGION)

INFO:root:The reduced shu_agn table provided contains 2437 sources.
INFO:root:After the probability cut at p_rf >= 0.940, 1899 sources are left in the shu_agn table


In [5]:
VHS_TABLE = fp.load_and_clean_vhs(REGION)

INFO:root:The vhs table provided contains 1547498 sources.
INFO:root:The reduced vhs table provided contains 291905 sources.


In [6]:
SWEEP_TABLE = fp.load_and_clean_sweep(REGION)

INFO:root:The following bricks are in the requested region for the sweep table:
['120m005-130p000']
INFO:root:The reduced sweep table provided contains 2421599 sources.


## Matching

Now that the tables are properly loaded (except for GALEX, which we'll acquire while matching), we can match them one by one.

In [11]:
# In my master thesis, I found the following to be a reasonable matching radius between sweep and the shu et al. agn table:
match_radius_shu = 0.1
MATCH_TABLE = fp.match_shu_with_sweep(SWEEP_TABLE, SHU_TABLE, match_radius=match_radius_shu)

INFO:root:Match of agn to sweep successful;
For 0 of the 1899 sources in the agn table, no match was found within the given radius.
INFO:root:From now on, the ra and dec columns refer to the sweep ra and dec.


1899


In [15]:
# In my master thesis, I found the following to be a reasonable matching radius to vhs:
match_radius_vhs = 0.19
MATCH_TABLE2 = fp.match_vhs_to_table(MATCH_TABLE, VHS_TABLE, match_radius=match_radius_vhs)

INFO:root:Found 683 matching vhs sources within the prescribed radius.


1899

In [49]:
# In my master thesis, I found the following to be a reasonable matching radius to galex:
match_radius_galex = 2.1
MATCH_TABLE3 = fp.match_with_galex_and_clean_it(MATCH_TABLE2, match_radius=match_radius_galex)

INFO:root:Found 526 matching galex sources within the prescribed radius.


In [50]:
# Write a backup of the matched table to disk
fp.write_table_as_backup(MATCH_TABLE3, "match_backup")

INFO:root:Successfully written a match_backup file at /hshome/bat3083/sel_4hi_q_v2/data/match_backups/base_full_match.fits.


## Processing

The matched table can now be processed.\
For this, all columns are turned into corrected magnitude ones, and the table is split up into a *pointlike* and an *extended* part.

In [51]:
MATCH_TABLE3 = fp.read_table_from_backup("match_backup")
PROCESSED = fp.process_galex_columns(MATCH_TABLE3)
PROCESSED = fp.process_sweep_columns(PROCESSED)
POINTLIKE, EXTENDED = fp.split_table_by_sourcetype(PROCESSED)
POINTLIKE = fp.process_vhs_columns(POINTLIKE)
EXTENDED = fp.process_vhs_columns(EXTENDED)
fp.write_table_as_backup(POINTLIKE, "processed_backup", "pointlike")
fp.write_table_as_backup(EXTENDED, "processed_backup", "extended")

/home/hslxrsrv3/bat3083/.local/lib/python3.8/site-packages/numpy/ma/core.py:6849: RuntimeWarning: overflow encountered in power
  result = np.where(m, fa, umath.power(fa, fb)).view(basetype)
INFO:root:Successfully written a processed_backup file at /hshome/bat3083/sel_4hi_q_v2/data/match_backups/base_processed_pointlike.fits.
INFO:root:Successfully written a processed_backup file at /hshome/bat3083/sel_4hi_q_v2/data/match_backups/base_processed_extended.fits.


## Create LePhare input files

Now that we do have the pointlike and extended subsets, we can create the LePhare input table:

In [53]:
POINTLIKE_LEPHARE = fp.process_for_lephare(POINTLIKE)
EXTENDED_LEPHARE = fp.process_for_lephare(EXTENDED)
fp.write_table_as_backup(POINTLIKE_LEPHARE, "lephare_in", "pointlike", overwrite=True)
fp.write_table_as_backup(EXTENDED_LEPHARE, "lephare_in", "extended", overwrite=True)

INFO:root:Successfully written a lephare_in file at /hshome/bat3083/sel_4hi_q_v2/data/lephare/input/base_input_pointlike.in.
INFO:root:Successfully written a lephare_in file at /hshome/bat3083/sel_4hi_q_v2/data/lephare/input/base_input_extended.in.
